# Week 2 Day 5 Exercise - Radio Africa Products Chatbot


This chatbot provides comprehensive information about Radio Africa Products, including:
- **Career Opportunities**: View and manage job openings
- **Radio Station Costs**: Get and set advertising costs for 5 radio stations
- **Database Integration**: Persistent storage with SQLite (ral.db)

### Radio Stations:
- **Kiss FM**: Kenya's leading urban radio station
- **Classic 105**: Kenya's premier classic hits station  
- **Radio Jambo**: Kenya's most popular vernacular station
- **Homeboyz Radio**: Kenya's youth-focused radio station
- **Gukena FM**: Kenya's leading vernacular station


In [1]:
# Import necessary libraries
import os
import json
import sqlite3
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [2]:
# Initialize OpenAI client
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# Database setup
DB = "ral.db"

# System message for the Radio Africa assistant
system_message = """
You are a helpful assistant for Radio Africa Products, a leading media company in Kenya.
You can provide information about:
- Career opportunities at Radio Africa
- Advertising costs for our 5 radio stations (Kiss FM, Classic 105, Radio Jambo, Homeboyz Radio, Gukena FM)
- Spot ad costs and sponsorship costs for each station
- General information about Radio Africa Products

Give helpful, accurate answers. If you don't know something, say so.
Keep responses concise but informative.
"""

print("✅ Radio Africa Products Assistant initialized!")


OpenAI API Key exists and begins sk-proj-
✅ Radio Africa Products Assistant initialized!


In [3]:
# Database setup
def setup_database():
    """Initialize the database with required tables"""
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        
        # Radio stations table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS radio_stations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT UNIQUE NOT NULL,
                spot_ad_cost REAL NOT NULL,
                sponsorship_cost REAL NOT NULL,
                description TEXT
            )
        ''')
        
        # Career opportunities table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS career_opportunities (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT NOT NULL,
                department TEXT NOT NULL,
                description TEXT,
                requirements TEXT,
                salary_range TEXT,
                location TEXT,
                is_active BOOLEAN DEFAULT 1
            )
        ''')
        
        conn.commit()
    print("✅ Radio Africa database setup complete!")

# Setup the database
setup_database()


✅ Radio Africa database setup complete!


In [4]:
# Tool functions
def get_radio_station_costs(station_name):
    """Get advertising costs for a specific radio station"""
    print(f"DATABASE TOOL CALLED: Getting costs for {station_name}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT name, spot_ad_cost, sponsorship_cost, description FROM radio_stations WHERE name LIKE ?', (f'%{station_name}%',))
        result = cursor.fetchone()
        if result:
            return f"Station: {result[0]}\nSpot Ad Cost: KSh {result[1]:,}\nSponsorship Cost: KSh {result[2]:,}\nDescription: {result[3]}"
        else:
            return f"No information found for {station_name}. Available stations: Kiss FM, Classic 105, Radio Jambo, Homeboyz Radio, Gukena FM"

def set_radio_station_costs(station_name, spot_ad_cost, sponsorship_cost):
    """Set advertising costs for a specific radio station"""
    print(f"DATABASE TOOL CALLED: Setting costs for {station_name}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            UPDATE radio_stations 
            SET spot_ad_cost = ?, sponsorship_cost = ?
            WHERE name LIKE ?
        ''', (spot_ad_cost, sponsorship_cost, f'%{station_name}%'))
        
        if cursor.rowcount > 0:
            conn.commit()
            return f"Successfully updated costs for {station_name}: Spot Ad - KSh {spot_ad_cost:,}, Sponsorship - KSh {sponsorship_cost:,}"
        else:
            return f"Station {station_name} not found. Available stations: Kiss FM, Classic 105, Radio Jambo, Homeboyz Radio, Gukena FM"

def get_career_opportunities(department=None):
    """Get career opportunities, optionally filtered by department"""
    print(f"DATABASE TOOL CALLED: Getting career opportunities for {department or 'all departments'}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        if department:
            cursor.execute('''
                SELECT title, department, description, requirements, salary_range, location 
                FROM career_opportunities 
                WHERE department LIKE ? AND is_active = 1
            ''', (f'%{department}%',))
        else:
            cursor.execute('''
                SELECT title, department, description, requirements, salary_range, location 
                FROM career_opportunities 
                WHERE is_active = 1
            ''')
        
        results = cursor.fetchall()
        if results:
            opportunities = []
            for row in results:
                opportunities.append(f"Title: {row[0]}\nDepartment: {row[1]}\nDescription: {row[2]}\nRequirements: {row[3]}\nSalary: {row[4]}\nLocation: {row[5]}\n")
            return "\n".join(opportunities)
        else:
            return f"No career opportunities found for {department or 'any department'}"

def add_career_opportunity(title, department, description, requirements, salary_range, location):
    """Add a new career opportunity"""
    print(f"DATABASE TOOL CALLED: Adding career opportunity - {title}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO career_opportunities (title, department, description, requirements, salary_range, location, is_active)
            VALUES (?, ?, ?, ?, ?, ?, 1)
        ''', (title, department, description, requirements, salary_range, location))
        conn.commit()
    return f"Successfully added career opportunity: {title} in {department}"

print("✅ Tool functions defined!")


✅ Tool functions defined!


In [5]:
# Tool definitions for OpenAI
get_radio_costs_function = {
    "name": "get_radio_station_costs",
    "description": "Get advertising costs (spot ad and sponsorship) for a specific radio station.",
    "parameters": {
        "type": "object",
        "properties": {
            "station_name": {
                "type": "string",
                "description": "The name of the radio station (Kiss FM, Classic 105, Radio Jambo, Homeboyz Radio, Gukena FM)",
            },
        },
        "required": ["station_name"],
        "additionalProperties": False
    }
}

set_radio_costs_function = {
    "name": "set_radio_station_costs",
    "description": "Set advertising costs for a specific radio station.",
    "parameters": {
        "type": "object",
        "properties": {
            "station_name": {
                "type": "string",
                "description": "The name of the radio station",
            },
            "spot_ad_cost": {
                "type": "number",
                "description": "The new spot ad cost",
            },
            "sponsorship_cost": {
                "type": "number",
                "description": "The new sponsorship cost",
            },
        },
        "required": ["station_name", "spot_ad_cost", "sponsorship_cost"],
        "additionalProperties": False
    }
}

get_careers_function = {
    "name": "get_career_opportunities",
    "description": "Get available career opportunities, optionally filtered by department.",
    "parameters": {
        "type": "object",
        "properties": {
            "department": {
                "type": "string",
                "description": "The department to filter by (optional)",
            },
        },
        "required": [],
        "additionalProperties": False
    }
}

add_career_function = {
    "name": "add_career_opportunity",
    "description": "Add a new career opportunity to the database.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {
                "type": "string",
                "description": "The job title",
            },
            "department": {
                "type": "string",
                "description": "The department",
            },
            "description": {
                "type": "string",
                "description": "Job description",
            },
            "requirements": {
                "type": "string",
                "description": "Job requirements",
            },
            "salary_range": {
                "type": "string",
                "description": "Salary range",
            },
            "location": {
                "type": "string",
                "description": "Job location",
            },
        },
        "required": ["title", "department", "description", "requirements", "salary_range", "location"],
        "additionalProperties": False
    }
}

# List of available tools
tools = [
    {"type": "function", "function": get_radio_costs_function},
    {"type": "function", "function": set_radio_costs_function},
    {"type": "function", "function": get_careers_function},
    {"type": "function", "function": add_career_function}
]

print("🔧 Tools configured:")
print(f"   - {get_radio_costs_function['name']}: {get_radio_costs_function['description']}")
print(f"   - {set_radio_costs_function['name']}: {set_radio_costs_function['description']}")
print(f"   - {get_careers_function['name']}: {get_careers_function['description']}")
print(f"   - {add_career_function['name']}: {add_career_function['description']}")


🔧 Tools configured:
   - get_radio_station_costs: Get advertising costs (spot ad and sponsorship) for a specific radio station.
   - set_radio_station_costs: Set advertising costs for a specific radio station.
   - get_career_opportunities: Get available career opportunities, optionally filtered by department.
   - add_career_opportunity: Add a new career opportunity to the database.


In [7]:
# Tool call handler
def handle_tool_calls(message):
    """Handle multiple tool calls from the LLM"""
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_radio_station_costs":
            arguments = json.loads(tool_call.function.arguments)
            station_name = arguments.get('station_name')
            result = get_radio_station_costs(station_name)
            responses.append({
                "role": "tool",
                "content": result,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "set_radio_station_costs":
            arguments = json.loads(tool_call.function.arguments)
            station_name = arguments.get('station_name')
            spot_ad_cost = arguments.get('spot_ad_cost')
            sponsorship_cost = arguments.get('sponsorship_cost')
            result = set_radio_station_costs(station_name, spot_ad_cost, sponsorship_cost)
            responses.append({
                "role": "tool",
                "content": result,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "get_career_opportunities":
            arguments = json.loads(tool_call.function.arguments)
            department = arguments.get('department')
            result = get_career_opportunities(department)
            responses.append({
                "role": "tool",
                "content": result,
                "tool_call_id": tool_call.id
            })
        elif tool_call.function.name == "add_career_opportunity":
            arguments = json.loads(tool_call.function.arguments)
            title = arguments.get('title')
            department = arguments.get('department')
            description = arguments.get('description')
            requirements = arguments.get('requirements')
            salary_range = arguments.get('salary_range')
            location = arguments.get('location')
            result = add_career_opportunity(title, department, description, requirements, salary_range, location)
            responses.append({
                "role": "tool",
                "content": result,
                "tool_call_id": tool_call.id
            })
    return responses

print("✅ Tool call handler configured!")


✅ Tool call handler configured!


In [8]:
# Main chat function
def chat(message, history):
    """Main chat function that handles tool calls"""
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # Handle tool calls in a loop to support multiple consecutive tool calls
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

print("✅ Chat function configured!")


✅ Chat function configured!


In [9]:
# Initialize sample data
def initialize_sample_data():
    """Initialize the database with sample data"""
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        
        # Clear existing data
        cursor.execute('DELETE FROM radio_stations')
        cursor.execute('DELETE FROM career_opportunities')
        
        # Insert radio stations data
        radio_stations = [
            ("Kiss FM", 15000, 500000, "Kenya's leading urban radio station"),
            ("Classic 105", 12000, 800000, "Kenya's premier classic hits station"),
            ("Radio Jambo", 10000, 1100000, "Kenya's most popular vernacular station"),
            ("Homeboyz Radio", 8000, 150000, "Kenya's youth-focused radio station"),
            ("Gukena FM", 6000, 100000, "Kenya's leading vernacular station")
        ]
        
        cursor.executemany('''
            INSERT INTO radio_stations (name, spot_ad_cost, sponsorship_cost, description)
            VALUES (?, ?, ?, ?)
        ''', radio_stations)
        
        # Insert career opportunities
        careers = [
            ("Radio Presenter", "Programming", "Host radio shows and engage with listeners", "Degree in Media/Communication, 2+ years experience", "KSh 80,000 - 150,000", "Nairobi", 1),
            ("Sales Executive", "Sales", "Generate advertising revenue and build client relationships", "Degree in Marketing/Business, 3+ years sales experience", "KSh 100,000 - 200,000", "Nairobi", 1),
            ("Content Producer", "Programming", "Create engaging radio content and manage social media", "Degree in Media/Journalism, 2+ years experience", "KSh 70,000 - 120,000", "Nairobi", 1),
            ("Technical Engineer", "Technical", "Maintain radio equipment and ensure smooth broadcasting", "Degree in Engineering, 3+ years technical experience", "KSh 90,000 - 160,000", "Nairobi", 1),
            ("Marketing Manager", "Marketing", "Develop marketing strategies and manage brand campaigns", "Degree in Marketing, 5+ years experience", "KSh 150,000 - 250,000", "Nairobi", 1),
            ("News Reporter", "News", "Research and report news stories for radio", "Degree in Journalism, 2+ years experience", "KSh 60,000 - 100,000", "Nairobi", 1),
            ("Digital Media Specialist", "Digital", "Manage digital platforms and online content", "Degree in Digital Media, 2+ years experience", "KSh 80,000 - 140,000", "Nairobi", 1)
        ]
        
        cursor.executemany('''
            INSERT INTO career_opportunities (title, department, description, requirements, salary_range, location, is_active)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', careers)
        
        conn.commit()
    print("✅ Sample data initialized!")

# Initialize sample data
initialize_sample_data()

# Test the setup
print("\n🧪 Testing the setup:")
print(get_radio_station_costs("Kiss FM"))
print("\n" + "="*50 + "\n")
print(get_career_opportunities("Sales"))


✅ Sample data initialized!

🧪 Testing the setup:
DATABASE TOOL CALLED: Getting costs for Kiss FM
Station: Kiss FM
Spot Ad Cost: KSh 15,000.0
Sponsorship Cost: KSh 500,000.0
Description: Kenya's leading urban radio station


DATABASE TOOL CALLED: Getting career opportunities for Sales
Title: Sales Executive
Department: Sales
Description: Generate advertising revenue and build client relationships
Requirements: Degree in Marketing/Business, 3+ years sales experience
Salary: KSh 100,000 - 200,000
Location: Nairobi



## Launch the Radio Africa Products Chatbot

The chatbot is now ready with comprehensive features for Radio Africa Products!


In [10]:
# Launch the Gradio interface
print("🚀 Launching Radio Africa Products Chatbot...")
print("📋 Available features:")
print("   - Get radio station advertising costs")
print("   - Set radio station advertising costs")
print("   - View career opportunities")
print("   - Add new career opportunities")
print("\n🎯 Example queries:")
print("   - 'What are the advertising costs for Kiss FM?'")
print("   - 'Show me career opportunities in Sales'")
print("   - 'Set the costs for Classic 105 to 15000 spot ads and 60000 sponsorship'")
print("   - 'What career opportunities are available?'")
print("   - 'Add a new job: Marketing Coordinator in Marketing department'")

interface = gr.ChatInterface(
    fn=chat, 
    type="messages",
    title="Radio Africa Products Assistant",
    description="Ask me about career opportunities, radio station costs, and Radio Africa Products!",
    examples=[
        "What are the advertising costs for Kiss FM?",
        "Show me career opportunities in Sales",
        "Set the costs for Classic 105 to 15000 spot ads and 60000 sponsorship",
        "What career opportunities are available?",
        "Add a new job: Marketing Coordinator in Marketing department"
    ]
)

interface.launch()


🚀 Launching Radio Africa Products Chatbot...
📋 Available features:
   - Get radio station advertising costs
   - Set radio station advertising costs
   - View career opportunities
   - Add new career opportunities

🎯 Example queries:
   - 'What are the advertising costs for Kiss FM?'
   - 'Show me career opportunities in Sales'
   - 'Set the costs for Classic 105 to 15000 spot ads and 60000 sponsorship'
   - 'What career opportunities are available?'
   - 'Add a new job: Marketing Coordinator in Marketing department'
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


DATABASE TOOL CALLED: Adding career opportunity - Marketing Coordinator


## Key Implementation Features

### 🎯 **Radio Station Management**
- **5 Radio Stations**: Kiss FM, Classic 105, Radio Jambo, Homeboyz Radio, Gukena FM
- **Cost Management**: Get and set spot ad costs and sponsorship costs
- **Station Information**: Descriptions and details for each station

### 💼 **Career Opportunities Management**
- **Job Listings**: View all available positions
- **Department Filtering**: Filter by specific departments (Sales, Programming, Technical, etc.)
- **Job Management**: Add new career opportunities
- **Detailed Information**: Job descriptions, requirements, salary ranges, locations

### 🗄️ **Database Schema (ral.db)**
```sql
-- Radio Stations Table
CREATE TABLE radio_stations (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE NOT NULL,
    spot_ad_cost REAL NOT NULL,
    sponsorship_cost REAL NOT NULL,
    description TEXT
);

-- Career Opportunities Table  
CREATE TABLE career_opportunities (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    department TEXT NOT NULL,
    description TEXT,
    requirements TEXT,
    salary_range TEXT,
    location TEXT,
    is_active BOOLEAN DEFAULT 1
);
```

### 🔧 **Tool Functions**
1. **get_radio_station_costs**: Query advertising costs for specific stations
2. **set_radio_station_costs**: Update advertising costs for stations
3. **get_career_opportunities**: View job opportunities (with optional department filter)
4. **add_career_opportunity**: Add new job postings

### 🚀 **Usage Examples**
- **Get Costs**: "What are the advertising costs for Kiss FM?"
- **Set Costs**: "Set the costs for Classic 105 to 15000 spot ads and 60000 sponsorship"
- **View Jobs**: "Show me career opportunities in Sales"
- **Add Jobs**: "Add a new job: Marketing Coordinator in Marketing department"

This implementation demonstrates comprehensive tool integration for a real-world business application!
